# 1.4: Exceptions, Copy, and Move Constructors

## Exceptions in C++

* exceptions provide a separate "flow of control"
* used for handling runtime errors "gracefully" (graceful degradation)
* *thrown* and *caught* (w/ values)
* for this current HW assignment, we will use it for index ouf of range errors

Here's an example of C++ code doing this:

In [ ]:
template<typename T>
void LinkedSeq<T>::insert(const T& elem, int index)
{
    if (index < 0 or index >= node_count) // could also say size()
    {
        throw std::out_of_range("LinkedSeq<T>::insert()");
    }
    
    // More function implementation
}

You should do that if statement above BEFORE WRITING ANYTHING ELSE IN YOUR CODE. 

Now if we try to write code like this:

In [ ]:
void some_other_function()
{
    LinkedSeq<char> s;
    
    try
    {
        s.insert('a',1); //caught exception
    }
    
    catch (std::out_of_range& ex)
    {
        cout << ex.what() << endl;
    }
    
    // More function implementation
}

## Essential Operators

There are various essential operators that we use in C++

* constructors (both default `X()` and overloaded `X(some params)`) 
* destructors (`~X()`)
* copy assignment (`X(const X& x)`)
* move assignment (`X(X&& x)`)
* copy assignment operator (`X& operator=(const X& x)`)
* move assignment operator (`X& operator=(X&& x)`)


> A **copy constructor** takes all of the values from `x2` and puts them into `x1`. 
>
> A **move constructor** takes all of the values from `x2`, *deletes them*, and puts them into `x1`.

Copy constructors can have problems because they tend to lead to *memory leaks* and *memory entanglement*

## Default Copy Assignment

In [ ]:
template<typename T>
LinkedSeq<T>& operator=(const LinkedSeq<T>& rhs)
{
    if (this != rhs&)
    {
        // delete lhs (using make_empty())
        // deep copy of rhs linked list
    }
    
    return *this;
}

// Copy constructor implementation
template<typename T>
LinkedSeq<T>(const LinkedSeq<T>& rhs)
{
    *this = rhs;
}

The copy constructor then becomes really easy because we just call the copy assignment operator.

## Move Assignment Operator

The move assignment operator helps the compiler avoid using excessive copies. For example, if we have this function:

```c++
LinkedSeq<int> add_one(const LinkedSeq<int>& s)
{
    LinkedSeq<int> temp;
    
    for (int i = 0; i < s.size(); ++i)
    {
        temp.insert(s[i] + 1, i);
    }
    
    return temp;
}
```

When we try and call this function to initialize a new `LinkedSeq<int>`, the compiler will recognize that there is no reason for the orignial `LinkedSeq<int>` to exist because we only care about the ouput and not the original sequence. Therefore we don't need to make a copy - we can just move everything from the original sequence to the new sequence. For can force move assignment by using the `std::move()` function.

The implementation of the move assignment operator is actually *simpler* that the copy assignment operator.

1. Transfer data from RHS to LHS
2. "zero-out" RHS (set pointers to `nullptr`)
3. **IF THIS IS ASSIGNMENT**, then clear out the LHS before transferring the RHS to LHS

The implementation is put below:

In [ ]:
// Move constructor
template<typename T>
LinkedSeq<T>& LinkedSeq<T>::LinkedSeq(LinkedSeq<T>&& rhs)
{
    *this = std::move(rhs);
}

// Move assignment operator
template<typename T>
LinekdSeq<T>& LinkedSeq<T>::operator=(LinkedSeq<T>&& rhs)
{
    if (this != &rhs)
    {
        //empty lhs
        make_empty(*this);
        
        //transfer
        head = rhs.head;
        tail = rhs.tail;
        node_count = rhs.node_count;
        
        //zero-out
        rhs.head = nullptr;
        rhs.tail = nullptr;
        node_count = 0;
    }
    
    return *this;
}